In [2]:
# c понедельничного семинара

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode


## Install NeMo
!python -m pip install nemo_toolkit[all]

!pip install torchtext==0.10.1
!pip install matplotlib==3.1.3
"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
# exit()

from IPython.display import clear_output
clear_output()

## Загрузка данных

In [2]:
data_dir = '/content'

In [3]:
import glob
import os
import subprocess
import tarfile
import wget

# Загрузка датасета Golos
def load(golos_url, fname):
  if not os.path.exists(os.path.join(data_dir, fname)):
      golos_path = wget.download(golos_url, os.path.join(data_dir, fname))
      print(f"Dataset downloaded at: {golos_path}")
  else:
      print("Tarfile already exists.")
      golos_path = os.path.join(data_dir, fname)
  return golos_path

### обучающая выборка

In [4]:
farfield_path = load("https://sc.link/1Z3", "farfield_train.tar")
if not os.path.exists(os.path.join(data_dir, '/farfield_train/')):
    tar = tarfile.open(farfield_path)
    tar.extractall(path=data_dir)

Dataset downloaded at: /content/farfield_train.tar


### тестовая выборка

In [5]:
golos_path = load("https://sc.link/Kqr", "test.tar")

if not os.path.exists(os.path.join(data_dir, '/test/')):
    tar = tarfile.open(golos_path)
    tar.extractall(path=data_dir)

Dataset downloaded at: /content/test.tar


### Манифесты

### достаем подгруженный манифет из репо

In [7]:
! unzip /content/train_manifest.jsonl.zip

Archive:  /content/train_manifest.jsonl.zip
  inflating: train_manifest.jsonl    


### убеждаемся что они все действительно существуют 

In [8]:
! head /content/train_manifest.jsonl

{"id": "77b380796d242cf5bc09cfb551cffecd", "audio_filepath": "crowd/0/77b380796d242cf5bc09cfb551cffecd.wav", "text": "\u0430\u043b\u0438\u0431\u0438", "duration": 1.21}
{"id": "404e1eecbc38a687e036739f2e5dd6d3", "audio_filepath": "crowd/0/404e1eecbc38a687e036739f2e5dd6d3.wav", "text": "\u0441\u043a\u0430\u0437\u043a\u0430", "duration": 2.9}
{"id": "f913db484595c1bcdd9476194b08a1f4", "audio_filepath": "crowd/0/f913db484595c1bcdd9476194b08a1f4.wav", "text": "\u043d\u0430\u0439\u0434\u0438 \u0432 \u043d\u0435\u0442\u0435 \u043c\u0443\u043b\u044c\u0442 \u0441\u0442\u0430\u043b\u044c\u043d\u043e\u0439 \u0433\u0438\u0433\u0430\u043d\u0442", "duration": 4.76}
{"id": "eca448acf1712fa7ca08aede3d2df1ac", "audio_filepath": "crowd/0/eca448acf1712fa7ca08aede3d2df1ac.wav", "text": "\u043c\u043e\u043d\u0430 \u043b\u0438\u0437\u0430", "duration": 2.3}
{"id": "a366eecb0a85ef76463e1f4b5188b698", "audio_filepath": "crowd/0/a366eecb0a85ef76463e1f4b5188b698.wav", "text": "\u0441\u043c\u043e\u0442\u0440\u04

In [9]:
! head /content/test/farfield/manifest.jsonl

{"id": "58b586f67f5e634506e215df5996b82e", "audio_filepath": "files/58b586f67f5e634506e215df5996b82e.wav", "text": "\u0434\u0436\u043e\u0439 \u0445\u0432\u0430\u0442\u0438\u0442", "duration": 1.696625}
{"id": "26093ef7a8c5ec6a3c586a6a929c1bd3", "audio_filepath": "files/26093ef7a8c5ec6a3c586a6a929c1bd3.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u0432\u044b\u0437\u043e\u0432 \u0441\u0432\u0435\u0442\u043b\u0430\u043d\u0435 \u0432\u0430\u0441\u0438\u043b\u044c\u0435\u0432\u043d\u0435 \u043d\u0438\u043a\u043e\u043b\u0435\u043d\u043a\u043e", "duration": 3.1720625}
{"id": "b42d13ab3dc159fac39a1120e8011f66", "audio_filepath": "files/b42d13ab3dc159fac39a1120e8011f66.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u0445\u0432\u0430\u0442\u0438\u0442", "duration": 1.5226875}
{"id": "7e55843e745a239b35212e2caba77239", "audio_filepath": "files/7e55843e745a239b35212e2caba77239.wav", "text": "\u0434\u0436\u043e\u0439 \u0437\u0432\u043e\u043d\u043e\u043a \u044e\u0440\u0438\u044e \u0438\u0432\u0430

### меняем пути

In [12]:
# --- Building Manifest Files --- #
import json
# Function to build a manifest
def build_manifest(manifest_rel, manifest_abs, test=False):
    n = 0
    manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
    with open(manifest_rel, 'r') as fin:
        with open(manifest_abs, 'w') as fout:
            for line in fin:
                n += 1
                if n < 20000 and not test:
                  continue
                if n > 50000:
                  break
                metadata = json.loads(line)
                if metadata['id'] == '375e547e51a71f9e18b811dd89244baf':
                  # print('empty')
                  continue
                metadata["audio_filepath"]=os.path.join(manifest_path, metadata["audio_filepath"])
                json.dump(metadata, fout)
                fout.write('\n')

In [10]:
# Building Manifests
print("******")
train_rel = os.path.join(data_dir, '/content/train_manifest.jsonl')
train_abs = os.path.join(data_dir, 'train/train_manifest.jsonl')
if not os.path.isfile(train_abs):
  build_manifest(train_rel, train_abs, False)
train_manifest = train_abs
print("train_manifest", train_manifest)

test_rel = os.path.join(data_dir, 'test/farfield/manifest.jsonl')
test_abs = os.path.join(data_dir, 'test/farfield/test_manifest.jsonl')
if not os.path.isfile(test_abs):
  build_manifest(test_rel, test_abs, True)
test_manifest = test_abs
print("test_manifest", test_manifest)

******
train_manifest /content/train/train_manifest.jsonl
test_manifest /content/test/farfield/test_manifest.jsonl


## Достаем модель

In [ ]:
import nemo
import nemo.collections.asr as nemo_asr

[NeMo W 2021-11-30 12:50:19 optimizers:50] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [ ]:
load("https://sc.link/ZMv", "QuartzNet15x5_golos.nemo")
model = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join(data_dir, "QuartzNet15x5_golos.nemo"))

Tarfile already exists.


[NeMo W 2021-11-29 12:49:16 modelPT:131] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2021-11-29 12:49:16 modelPT:138] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2021-11-29 12:49:16 features:265] PADDING: 16
[NeMo I 2021-11-29 12:49:16 features:282] STFT using torch
[NeMo I 2021-11-29 12:49:17 save_restore_connector:149] Model EncDecCTCModel was successfully restored from /content/QuartzNet15x5_golos.nemo.


### настраиваем параметры

In [ ]:
import copy
from omegaconf import DictConfig

batch_size = 16

params_model = model._cfg
params_model['train_ds']['parser'] = 'base'
params_model['validation_ds']['parser'] = 'base'
params_model['test_ds']['parser'] = 'base'

params_model['train_ds']['manifest_filepath'] = train_manifest
params_model['train_ds']['batch_size'] = batch_size

params_model['test_ds']['manifest_filepath'] = test_manifest
params_model['test_ds']['batch_size'] = batch_size

params_model['validation_ds']['manifest_filepath'] = test_manifest
params_model['validation_ds']['batch_size'] = batch_size

In [ ]:
model.setup_training_data(train_data_config=params_model['train_ds'])
model.setup_test_data(test_data_config=params_model['test_ds'])
model.setup_validation_data(val_data_config=params_model['validation_ds'])


lr = 0.0005

params_optim = copy.deepcopy(params_model['optim'])
params_optim['lr'] = lr
model.setup_optimization(optim_config=DictConfig(params_optim))

[NeMo I 2021-11-29 12:49:33 collections:173] Dataset loaded with 29999 files totalling 31.31 hours
[NeMo I 2021-11-29 12:49:33 collections:174] 1 files were filtered totalling 0.00 hours


[NeMo W 2021-11-29 12:49:33 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-29 12:49:34 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-29 12:49:34 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-29 12:49:34 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-29 12:49:34 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2021-11-29 12:49:34 modelPT:453] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-29 12:49:34 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0005
        weight_decay: 0.001
    )


[NeMo W 2021-11-29 12:49:34 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


In [8]:
# Bigger batch-size = bigger throughput
# params['model']['validation_ds']['batch_size'] = 16
# Setup the test data loader and make sure the model is on GPU
# asr_model.setup_test_data(test_data_config=params['model']['validation_ds'])
model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = model._wer.compute()
        model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")


print(f'WER до дообучения составляет {sum(wer_nums)/sum(wer_denoms)}')


WER до дообучения составляет 0.6985648962206105


### Обучаем

In [ ]:
import pytorch_lightning as pl
trainer = pl.Trainer(gpus=1, max_epochs=1, precision=16)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-11-29 12:49:43 modelPT:453] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-29 12:49:43 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0005
        weight_decay: 0.001
    )


[NeMo W 2021-11-29 12:49:43 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
37.859    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-29 12:49:44 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    
[NeMo W 2021-11-29 12:49:46 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

### Считаем метрику

In [9]:
# Bigger batch-size = bigger throughput
# params['model']['validation_ds']['batch_size'] = 16
# Setup the test data loader and make sure the model is on GPU
# asr_model.setup_test_data(test_data_config=params['model']['validation_ds'])
model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = model._wer.compute()
        model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

WER = 0.5802061062856485
